![title](img/logo.png)
## Exercise: Clustering of artists
===

In this exercise, we will use a real world music dataset from [Last.fm](http://last.fm) to experience with Unsupervised Clustering methods. 

Note: The play data (and user/artist matrix) comes from the [Last.fm 1K Users dataset](http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/lastfm-1K.html), while the tags come from [the Last.fm Music Tags dataset](http://musicmachinery.com/2010/11/10/lastfm-artisttags2007/). You won't have to interact with these datasets directly, because we've already preprocessed them for you.

### Files

Data files for this assignment can be found at: `/volumes/data/lastfm/`

The folder includes the following files:

* **artists-tags.txt**, User-defined tags for top artists
* **userart-mat-training.csv**, Training data containing a matrix mapping artist-id to users who have played songs by the artists
* **userart-mat-test.csv**, Test data containing a matrix mapping artist-id to users who have played songs by the artists
* **train_model_data.csv**, Aggregate statsitstics and features about songs we'll use to train regression models.
* **validation_model_data.csv**, Similar statistics computed on a hold-out set of users and songs that we'll use to validate our regression models.

We will explain the datasets and how they need to used later.

## Part 0: Preliminaries

### Exercise 0

Read in the file **artists-tags.txt** and store the contents in a Pandas DataFrame. The file format for this file is `artist-id|artist-name|tag|count`. The fields mean the following:

1. artist-id : a unique id for an artist (Formatted as a [MusicBrainz Identifier](https://musicbrainz.org/doc/MusicBrainz_Identifier))
2. artist-name: name of the artist
3. tag: user-defined tag for the artist
4. count: number of times the tag was applied

Similarly, read in the file **userart-mat-training.csv** . The file format for this file is `artist-id, user1, user2, .... user1000`. i.e. There are 846 such columns in this file and each column has a value 1 if the particular user played a song from this artist.

In [3]:
import pandas as pd

DATA_PATH = "/volumes/data/lastfm"

def parse_artists_tags(filename):
    df = pd.read_csv(filename, sep="|", names=["ArtistID", "ArtistName", "Tag", "Count"])
    return df

def parse_user_artists_matrix(filename):
    df = pd.read_csv(filename)
    return df

artists_tags = parse_artists_tags(DATA_PATH + "/artists-tags.txt")
user_art_mat = parse_user_artists_matrix(DATA_PATH + "/userart-mat-training.csv")

print "Number of tags %d" % len(artists_tags.Tag.unique()) # Change this line. Should be 952803
print "Number of artists %d" % len(user_art_mat) # Change this line. Should be 17119

Number of tags 100782
Number of artists 17119


## Part 1: Finding genres by clustering

The first task we will look at is how to discover artist genres by only looking at data from plays on Last.fm. One of the ways to do this is to use clustering. To evaluate how well our clustering algorithm performs we will use the user-generated tags and compare those to our clustering results. 

### 1.1 Data pre-processing

Last.fm allows users to associate tags with every artist (See the [top tags](http://www.last.fm/charts/toptags) for a live example). However as there are a number of tags associated with every artists, in the first step we will pre-process the data and get the most popular tag for an artist.

#### Exercise 1

**a**. For every artist in **artists_tags** calculate the most frequently used tag. 

In [4]:
artists_tags.head()

,ArtistID,ArtistName,Tag,Count
0,000077f7-26b1-4710-80cc-f6beddbdd157,Ryan Adams and The Cardinals,I love you baby can I have some more,1
1,000077f7-26b1-4710-80cc-f6beddbdd157,Ryan Adams and The Cardinals,alt country,2
2,000077f7-26b1-4710-80cc-f6beddbdd157,Ryan Adams and The Cardinals,whoa,1
3,00034ede-a1f1-4219-be39-02f36853373e,O Rappa,Artist,1
4,00034ede-a1f1-4219-be39-02f36853373e,O Rappa,Black,1


In [5]:
user_art_mat.head()

,ArtistID,user_000001,user_000002,user_000003,user_000004,user_000005,user_000006,user_000007,user_000008,user_000009,...,user_000890,user_000891,user_000892,user_000893,user_000894,user_000895,user_000896,user_000897,user_000898,user_000899
0,c489cd1c-d8d3-4dcc-b7b9-dcf7cd95d2ed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9ccbb935-33fd-4df8-ae2d-779497c5630a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,c71b2cc1-2737-4f76-bbf2-9ee505b4d90d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0aa8294b-6332-4b65-b677-e3a1f8591d3b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,67673557-2310-41d3-83b9-e6e0cb1d65d5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for r in artists_tags.iterrows():
    row = r[1]
    print row['ArtistID']

000077f7-26b1-4710-80cc-f6beddbdd157
000077f7-26b1-4710-80cc-f6beddbdd157
000077f7-26b1-4710-80cc-f6beddbdd157
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
00034ede-a1f1-4219-be39-02f36853373e
0

In [11]:
artists_tags.groupby('ArtistID').apply(lambda t: t[t.Count == t.Count.max()][['ArtistName', 'Tag']].head(1))\
.reset_index()[['ArtistID', 'ArtistName', 'Tag']]

,ArtistID,ArtistName,Tag
0,000077f7-26b1-4710-80cc-f6beddbdd157,Ryan Adams and The Cardinals,alt country
1,00034ede-a1f1-4219-be39-02f36853373e,O Rappa,rock
2,00050add-f633-4901-8d93-6f88c640c0da,9 Inch Dix,rap
3,000b1990-4dd8-4835-abcd-bb6038c13ac7,Hayden,indie
4,000ba849-700e-452e-8858-0db591587e4a,The Mutton Birds,New Zealand
5,000d90ec-d64c-48a1-b775-e726fd240e9f,Get Cape. Wear Cape. Fly,acoustic
6,000fc734-b7e1-4a01-92d1-f544261b43f5,Cocteau Twins,shoegaze
7,0019749d-ee29-4a5f-ab17-6bfa11deb969,DJ Food,ninja tune
8,001aca82-d3bf-4a02-afd3-297740d12c14,David Dondero,seen live
9,001ce2d7-c045-4343-b703-a4fc7dcee0a6,Gravy Train!!!!,dance


In [13]:
# TODO Implement this. You can change the function arguments if necessary
# Return a data structure that contains (artist id, artist name, top tag) for every artist
def calculate_top_tag(all_tags):
    return artists_tags.groupby('ArtistID').apply(lambda t: t[t.Count == t.Count.max()][['ArtistName', 'Tag']].head(1))\
.reset_index()[['ArtistID', 'ArtistName', 'Tag']]

top_tags = calculate_top_tag(artists_tags)

# Print the top tag for Nirvana
# Artist ID for Nirvana is 5b11f4ce-a62d-471e-81fc-a69a8278c7da
# Should be 'Grunge'
print "Top tag for Nirvana is %s" % top_tags[top_tags.ArtistID == '5b11f4ce-a62d-471e-81fc-a69a8278c7da']['Tag'].item() # Complete this line 

Top tag for Nirvana is 7467    Grunge
Name: Tag, dtype: object


**b**. To do clustering we will be using `numpy` matrices. Create a matrix from **user_art_mat** with every row in the matrix representing a single artist. The matrix will have 846 columns, one for whether each user listened to the artist.

In [14]:
def create_user_matrix(input_data):
    return input_data[input_data.columns[1:]].values

user_np_matrix = create_user_matrix(user_art_mat)

print user_np_matrix.shape # Should be (17119, 846)

(17119, 846)


### 1.2 K-Means clustering

Having pre-processed the data we can now perform clustering on the dataset. In this assignment we will be using the python library 
[scikit-learn](http://scikit-learn.org/stable/index.html) for our machine learning algorithms. scikit-learn provides an extensive
library of machine learning algorithms that can be used for analysis. Here is a [nice flow chart](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) that shows various algorithms implemented
and when to use any of them. In this part of the assignment we will look at K-Means clustering

> **Note on terminology**: "samples" and "features" are two words you will come across frequently when you look at machine learning papers or documentation. "samples" refer to data points that are used as inputs to the machine learning algorithm. For example in our dataset each artist is a "sample". "features" refers to some representation we have for every sample. For example the list of 1s and 0s we have for each artist are "features". Similarly the bag-of-words approach from the previous homework produced "features" for each document.

#### K-Means algorithm

Clustering is the process of automatically grouping data points that are similar to each other. In the [K-Means algorithm](http://en.wikipedia.org/wiki/K-means_clustering) we start with `K` initially chosen cluster centers (or centroids). We then compute the distance of every point from the centroids and assign each point to the centroid. Next we update the centroids by averaging all the points in the cluster. Finally, we repeat the algorithm until the cluster centers are stable.

### Running K-Means

#### K-Means interface
Take a minute to look at the scikit-learn interface for calling [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). The constructor of the KMeans class returns a `estimator` on which you can call [fit](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans.fit) to perform clustering.

#### K-Means parameters
From the above description we can see that there are a few parameters which control the K-Means algorithm. We will look at one parameter specifically, the number of clusters used in the algorithm. The number of clusters needs to be chosen based on domain knowledge of the data. As we do not know how many genres exist we will try different values and compare the results.

#### Timing your code
We will also measure the performance of clustering algorithms in this section. You can time the code in a cell using the **%%time** [IPython magic](http://nbviewer.ipython.org/github/ipython/ipython/blob/1.x/examples/notebooks/Cell%20Magics.ipynb) as the first line in the cell. 

>**Note**: By default, the scikit-learn KMeans implementation runs the algorithm 10 times with different center initializations. For this assignment you can run it just once by passing the `n_init` argument as 1.

#### Exercise 2

**a**. Run K-means using *5* cluster centers on the `user_np_matrix`.

In [15]:
%%time
from sklearn.cluster import KMeans

# Run K-means using 5 cluster centers on user_np_matrix
kmeans_5 = KMeans(n_clusters=5, n_init=1)

kmeans_5.fit(user_np_matrix)

CPU times: user 27.6 s, sys: 284 ms, total: 27.9 s
Wall time: 27.9 s


**b**. Run K-means using *25* and *50* cluster centers on the `user_np_matrix`. Also measure the time taken for both cases.

In [ ]:
%%time
kmeans_25 = None

In [ ]:
%%time
kmeans_50 = None

### 1.3 Evaluating K-Means

In addition to the performance comparisons we also wish to compare how good our clusters are. To do this we are first going to look at internal evaluation metrics. For internal evaluation we only use the input data and the clusters created and try to measure the quality of clusters created. We are going to use two metrics for this:

#### Inertia
Inertia is a metric that is used to estimate how close the data points in a cluster are. This is calculated as the sum of squared distance for each point to it's closest centroid, i.e., its assigned cluster center. The intution behind inertia is that clusters with lower inertia are better as it means closely related points form a cluster.Inertia is calculated by scikit-learn by default.


**Exercise 3**

**a**. Print inertia for all the kmeans model computed above.

In [ ]:
print "Inertia for KMeans with 5 clusters = %lf " % 0.0
print "Inertia for KMeans with 25 clusters =  %lf " % 0.0
print "Inertia for KMeans with 50 clusters = %lf " % 0.0

**b**. Does KMeans run with 25 clusters have lower or greater inertia than the ones with 5 clusters ? Which algorithm is better and why ?

>TODO: Answer question

#### Silhouette Score: 
The silhouette score measures how close various clusters created are. A higher silhouette score is better as it means that we dont have too many overlapping clusters. The silhouette score can be computed using [sklearn.metrics.silhouette_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score) from scikit learn.

**c.** Calculate the Silhouette Score using 500 sample points for all the kmeans models.

In [ ]:
from sklearn.metrics import silhouette_score

# NOTE: Use 500 sample points to calculate the silhouette score
def get_silhouette_score(data, model):
    pass

print "Silhouette Score for KMeans with 5 clusters = %lf" % 0.0
print "Silhouette Score for KMeans with 25 clusters = %lf " % 0.0
print "Silhouette Score for KMeans with 50 clusters = %lf " % 0.0

### 1.4 External Evaluation
While internal evaluation is useful, a better method for measuring clustering quality is to do external evaluation. This might not be possible always as we may not have ground truth data available. In our application we will use `top_tags` from before as our ground truth data for external evaluation. We will first compute purity and accuracy and finally we will predict tags for our **test** dataset.

#### Exercise 4

**a**. As a first step we will need to **join** the `artist_tags` data with the set of labels generated by K-Means model. That is, for every artist we will now have the top tag, cluster id and artist name in a data structure.

In [ ]:
# Return a data structure that contains artist_id, artist_name, top tag, cluster_label for every artist
def join_tags_labels(artists_data, user_data, kmeans_model):
    pass
    
# Run the function for all the models
kmeans_5_joined = None
kmeans_25_joined = None
kmeans_50_joined = None

In [ ]:
kmeans_5_joined.head()

**b**. Next we need to generate a genre for every cluster id we have (the cluster ids are from 0 to N-1). You can do this by **grouping** the data from the previous exercise on cluster id. 

One thing you might notice is that we typically get a bunch of different tags associated with every cluster. How do we pick one genre or tag from this ? To cover various tags that are part of the cluster, we will pick the **top 5** tags in each cluster and save the list of top-5 tags as the genre for the cluster.


In [ ]:
# Return a data structure that contains cluster_id, list of top 5 tags for every cluster
def assign_cluster_tags(joined_data):
    pass
    
    
kmeans_5_genres = None
kmeans_25_genres = None
kmeans_50_genres = None

#### Purity and Accuracy
Two commonly used metrics used for evaluating clustering using external labels are purity and accuracy. **Purity** measures the frequency of data belonging to the same cluster sharing the same class label i.e. if we have a number of items in a cluster how many of those items have the same label ? Meanwhile, **accuracy** measures the frequency of data from the same class appearing in a single cluster i.e. of all the items which have a particular label what fraction appear in the same cluster ?


**d**. Compute the purity for each of our K-Means models. To do this find the top tags of all artists that belong to a cluster. Check what fraction of these tags are covered by the top 5 tags of the cluster. Average this value across all clusters. **HINT**: We used similar ideas to get the top 5 tags in a cluster. 

In [ ]:
def get_cluster_purity(joined_data):
    pass

print "Purity for KMeans with 5 centers %lf " % 0.0
print "Purity for KMeans with 25 centers %lf " % 0.0
print "Purity for KMeans with 50 centers %lf " % 0.0

**e**. To compute the accuracy first get all the unique tags from *top_tags*. Then for each tag, compute how many artists are found in the largest cluster. We denote these as correct cluster assignments. For example, lets take a tag 'rock'. If there are 100 artists with tag 'rock' and say 90 of them are in one cluster while 10 of them are in another. Then we have 90 correct cluster assignments

Add the number of correct cluster assignments for all tags and divide this by the total size of the training data to get the accuracy for a model.

In [ ]:
def get_accuracy(joined_data):
    pass
    
print "Accuracy of KMeans with 5 centers %lf " % 0.0
print "Accuracy of KMeans with 25 centers %lf " % 0.0
print "Accuracy of KMeans with 50 centers %lf " % 0.0

**f.** What do the numbers tell you about the models? Do you have a favorite?

>TODO: Your answer here.

### 1.5 Evaluating Test Data
Finally we can treat the clustering model as a multi-class classifier and make predictions on external test data. To do this we load the test data file **userart-mat-test.csv** and for every artist in the file we use the K-Means model to predict a cluster. We mark our prediction as successful if the artist's top tag belongs to one of the five tags for the cluster. 

#### Exercise 5

**a** Load the testdata file and create a NumPy matrix named user_np_matrix_test.

In [ ]:
user_np_matrix_test.shape # Should be (1902, 846)

**b.** For each artist in the test set, call **[predict](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans.predict)** to get the predicted cluster. Join the predicted labels with test artist ids. Return 'artist_id', 'predicted_label' for every artist in the test dataset.

In [ ]:
# For every artist return a list of labels
def predict_cluster(test_data, test_np_matrix, kmeans_model):
    return None

# Call the function for every model from before
kmeans_5_predicted = predict_cluster(artists_tags, user_art_mat_test, kmeans_5)
kmeans_25_predicted = predict_cluster(artists_tags, user_art_mat_test, kmeans_25)
kmeans_50_predicted = predict_cluster(artists_tags, user_art_mat_test, kmeans_50)

**c**. Get the tags for the predicted genre and the tag for the artist from `top_tags`. Output the percentage of artists for whom the top tag is one of the five that describe its cluster. This is the *recall* of our model.
>NOTE: Since the tag data is not from the same source as user plays, there are artists in the test set for whom we do not have top tags. You should exclude these artists while making predictions and while computing the recall.

**d**. Print the recall for each KMeans model. We define recall as num_correct_predictions / num_artists_in_test_data

In [ ]:
# Use verify_predictions for every model
print "Recall of KMeans with 5 centers %lf " % verify_predictions(kmeans_5_predicted, kmeans_5_genres, top_tags )
print "Recall of KMeans with 25 centers %lf " % verify_predictions(kmeans_25_predicted, kmeans_25_genres, top_tags )
print "Recall of KMeans with 50 centers %lf " % verify_predictions(kmeans_50_predicted, kmeans_50_genres, top_tags )

### 1.5 Visualizing Clusters using PCA

Another way to evaluate clustering is to visualize the output of clustering. However the data we are working with is in 846 dimensions !, so it is hard to visualize or plot this. Thus the first step for visualization is to reduce the dimensionality of the data. To do this we can use [Prinicipal Component Analysis (PCA)](http://en.wikipedia.org/wiki/Principal_component_analysis). PCA reduces the dimension of data and keeps only the most significant components of it. This is a commonly used technique to visualize data from high dimensional spaces.

>**NOTE**: We use [RandomizedPCA](http://scikit-learn.org/stable/modules/decomposition.html#approximate-pca), an approximate version of the algorithm as this has lower memory requirements. The approximate version is good enough when we are reducing to a few dimensions (2 in this case). We also sample the input data before PCA to further reduce memory requirements.

#### Exercise 6

**a**. Calcluate the RandomizedPCA of the sampled training data set `sampled_data` and reduce it to 2 components. Use the [fit_transform](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.RandomizedPCA.html#sklearn.decomposition.RandomizedPCA.fit_transform) method to do this.

In [ ]:
from sklearn.decomposition import RandomizedPCA
import numpy as np

input_data = user_np_matrix

sample_percent = 0.50
rows_to_sample = int(np.ceil(sample_percent * user_np_matrix.shape[0]))
sampled_data = input_data[np.random.choice(input_data.shape[0], rows_to_sample, replace=False),:]

# Return the data reduced to 2 principal components
def get_reduced_data(input_data):
    return None

user_np_2d = get_reduced_data(sampled_data)

**b**. Fit the reduced data with the KMeans model with 5 cluster centers. Plot the cluster centers and all the points. Make sure to color points in every cluster differently to see a visual separation. You may find [`scatter`](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.scatter) and [`plot`](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.plot) functions from matplotlib to be useful.


In [ ]:
# TODO: Write code to fit and plot reduced_data.
